## Deliverable 2. Create a Customer Travel Destinations Map.

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,Vaini,-21.2000,-175.2000,77.16,73,20,12.66,few clouds,TO
1,1,Bayir,37.2687,28.2167,64.92,45,3,1.48,clear sky,TR
2,2,Jamestown,42.0970,-79.2353,74.52,49,1,4.61,clear sky,US
3,3,Acapulco,16.8634,-99.8901,87.62,70,75,4.61,broken clouds,MX
4,4,Dunedin,-45.8742,170.5036,55.80,63,52,14.00,broken clouds,NZ


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,Vaini,-21.2000,-175.2000,77.16,73,20,12.66,few clouds,TO
2,2,Jamestown,42.0970,-79.2353,74.52,49,1,4.61,clear sky,US
3,3,Acapulco,16.8634,-99.8901,87.62,70,75,4.61,broken clouds,MX
5,5,Kahului,20.8947,-156.4700,87.82,63,40,23.02,scattered clouds,US
6,6,Airai,-8.9266,125.4092,65.43,38,32,4.56,scattered clouds,TL
7,7,Avarua,-21.2078,-159.7750,80.65,69,40,12.66,scattered clouds,CK
10,10,Rikitea,-23.1203,-134.9692,73.96,76,4,11.83,clear sky,PF
13,13,Katsuura,35.1333,140.3000,69.91,69,86,16.64,overcast clouds,JP
17,17,Hunsur,12.3075,76.2878,68.63,93,95,4.54,overcast clouds,IN
18,18,Boa Vista,2.8197,-60.6733,80.58,89,75,4.61,broken clouds,BR


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                389
City                   389
Lat                    389
Lng                    389
Max Temp               389
Humidity               389
Cloudiness             389
Wind Speed             389
Current Description    389
Country                386
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                386
City                   386
Lat                    386
Lng                    386
Max Temp               386
Humidity               386
Cloudiness             386
Wind Speed             386
Current Description    386
Country                386
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,
2,Jamestown,US,74.52,clear sky,42.0970,-79.2353,
3,Acapulco,MX,87.62,broken clouds,16.8634,-99.8901,
5,Kahului,US,87.82,scattered clouds,20.8947,-156.4700,
6,Airai,TL,65.43,scattered clouds,-8.9266,125.4092,
7,Avarua,CK,80.65,scattered clouds,-21.2078,-159.7750,
10,Rikitea,PF,73.96,clear sky,-23.1203,-134.9692,
13,Katsuura,JP,69.91,overcast clouds,35.1333,140.3000,
17,Hunsur,IN,68.63,overcast clouds,12.3075,76.2878,
18,Boa Vista,BR,80.58,broken clouds,2.8197,-60.6733,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    #print(params)
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = None
        print("Hotel not found... skipping " + hotel_df.loc[index, "City"])   

print("<----------------->")        
print("Updates Complete!")        
print("<----------------->")

Hotel not found... skipping Albany
Hotel not found... skipping Caxito
Hotel not found... skipping Latung
Hotel not found... skipping Aripuana
Hotel not found... skipping Yaan
Hotel not found... skipping Kasamwa
Hotel not found... skipping Bukama
Hotel not found... skipping Marabba
Hotel not found... skipping Wagar
Hotel not found... skipping Sidi Ali
Hotel not found... skipping Ler
Hotel not found... skipping Sekoma
Hotel not found... skipping Marzuq
Hotel not found... skipping Merauke
Hotel not found... skipping Tambacounda
Hotel not found... skipping Balaipungut
Hotel not found... skipping San Lazaro
Hotel not found... skipping Abu Samrah
Hotel not found... skipping Venezuela
Hotel not found... skipping Micheweni
Hotel not found... skipping Thinadhoo
Hotel not found... skipping Avera
<----------------->
Updates Complete!
<----------------->


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.info()
hotel_df = hotel_df.dropna()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 386 entries, 0 to 697
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 386 non-null    object 
 1   Country              386 non-null    object 
 2   Max Temp             386 non-null    float64
 3   Current Description  386 non-null    object 
 4   Lat                  386 non-null    float64
 5   Lng                  386 non-null    float64
 6   Hotel Name           364 non-null    object 
dtypes: float64(3), object(4)
memory usage: 32.2+ KB


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Whether Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))